# 🚀 한국어-영어 번역 Transformer (개선된 버전)

## 주요 개선 사항
| 항목 | 기존 | 개선 |
|------|------|------|
| d_model | 256 | **512** |
| ffn_dim | 1024 | **2048** |
| Learning Rate | 고정 0.0005 | **Warmup Scheduler** |
| Epochs | 30 | **50** |
| Early Stopping | 3 | **7** |
| 번역 방식 | Greedy | **Beam Search 추가** |

---

# 1. 환경 설정 및 라이브러리 설치

In [ ]:
# 필요한 라이브러리 설치
!pip install -q sentencepiece easydict koeda gensim

# 기본 라이브러리
import os
import re
import math
import time
import random
from typing import List, Tuple, Dict

# 데이터 처리
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import nltk
try:
    from nltk.corpus import wordnet
    wordnet.synsets('test')
except LookupError:
    print("WordNet 다운로드 중...")
    nltk.download('wordnet', quiet=True)
    nltk.download('omw-1.4', quiet=True)

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

# SentencePiece
import sentencepiece as spm

# 시각화
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# 평가
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# 진행바
from tqdm import tqdm

# 설정
from easydict import EasyDict

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

# 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# 랜덤 시드 설정
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)

print("✅ 환경 설정 완료!")

# 2. 디렉토리 및 경로 설정

In [ ]:
# 작업 디렉토리 설정
dataset_dir = '/content/work/dataset'
os.makedirs(dataset_dir, exist_ok=True)

# 데이터 경로
DATA_DIR = '/content/work/dataset'
ENG_TRAIN_PATH = f'{DATA_DIR}/augmented_train.en'
KOR_TRAIN_PATH = f'{DATA_DIR}/augmented_train.ko'

# 저장 경로
MODEL_DIR = '/content/work/models'
TOKENIZER_DIR = '/content/work/tokenizers'
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(TOKENIZER_DIR, exist_ok=True)

MODEL_SAVE_PATH = f'{MODEL_DIR}/transformer_improved.pt'
ENG_TOKENIZER_PREFIX = f'{TOKENIZER_DIR}/kor_eng'
KOR_TOKENIZER_PREFIX = f'{TOKENIZER_DIR}/eng_kor'

print("✅ 경로 설정 완료!")
print(f"데이터 디렉토리: {DATA_DIR}")
print(f"모델 저장 경로: {MODEL_SAVE_PATH}")

# 3. ⭐ 개선된 설정 (핵심 변경!)

In [ ]:
# 데이터 설정
DATA_CONFIG = EasyDict({
    'train_ratio': 0.8,
    'val_ratio': 0.1,
    'test_ratio': 0.1,
    'max_length': 60,
    'vocab_size_eng': 8000,
    'vocab_size_kor': 8000,
    'batch_size': 64,          # 128 → 64 (모델 크기 증가로 메모리 절약)
    'num_workers': 0,
})

# ⭐ 개선된 모델 설정
MODEL_CONFIG = EasyDict({
    'emb_dim': 512,            # 256 → 512 (핵심 개선!)
    'ffn_dim': 2048,           # 1024 → 2048 (핵심 개선!)
    'num_heads': 8,
    'encoder_layers': 6,
    'decoder_layers': 6,
    'dropout': 0.1,
    'attention_dropout': 0.1,
    'max_position_embeddings': 128,
})

# ⭐ 개선된 학습 설정
TRAIN_CONFIG = EasyDict({
    'num_epochs': 50,          # 30 → 50 (더 많은 학습)
    'warmup_steps': 4000,      # 핵심 추가! Warmup steps
    'betas': (0.9, 0.98),
    'eps': 1e-9,
    'weight_decay': 0.0001,    # 0.01 → 0.0001 (낮춤)
    'clip': 1.0,
    'label_smoothing': 0.1,
    'patience': 7,             # 3 → 7 (더 인내심 있게)
    'min_delta': 1e-4,
})

print("=" * 60)
print("✅ 개선된 설정 완료!")
print("=" * 60)
print(f"모델 크기: d_model={MODEL_CONFIG.emb_dim}, ffn_dim={MODEL_CONFIG.ffn_dim}")
print(f"배치 크기: {DATA_CONFIG.batch_size}")
print(f"Warmup Steps: {TRAIN_CONFIG.warmup_steps}")
print(f"총 Epochs: {TRAIN_CONFIG.num_epochs}")
print(f"Early Stopping Patience: {TRAIN_CONFIG.patience}")
print("=" * 60)

# 4. 데이터 다운로드 및 전처리

In [ ]:
# 데이터 다운로드
print("데이터 다운로드 중...")
!wget -q https://github.com/jungyeul/korean-parallel-corpora/raw/master/korean-english-news-v1/korean-english-park.train.tar.gz -P {dataset_dir}
!gzip -df {dataset_dir}/korean-english-park.train.tar.gz 2>/dev/null || true
!tar -xf {dataset_dir}/korean-english-park.train.tar -C {dataset_dir} 2>/dev/null || true

# 원본 데이터 경로
kor_path = os.path.join(dataset_dir, "korean-english-park.train.ko")
eng_path = os.path.join(dataset_dir, "korean-english-park.train.en")

# 데이터 로드
with open(kor_path, 'r', encoding='utf-8') as f:
    questions = [line.strip() for line in f.readlines()]

with open(eng_path, 'r', encoding='utf-8') as f:
    answers = [line.strip() for line in f.readlines()]

print(f"\n다운로드 완료!")
print(f"한국어 문장 개수: {len(questions):,}")
print(f"영어 문장 개수: {len(answers):,}")
print(f"\n예시:")
for i in range(2):
    print(f"Kor: {questions[i][:50]}...")
    print(f"Eng: {answers[i][:50]}...")
    print("-" * 50)

In [ ]:
# 전처리 함수
def preprocess_sentence(sentence: str, lang: str = 'kor') -> str:
    sentence = sentence.strip()
    if lang == 'eng':
        sentence = sentence.lower()
    sentence = re.sub(r"\s+", " ", sentence)
    return sentence.strip()

def preprocess_data_v2(eng_sentences, kor_sentences, show_samples=2):
    """강화된 전처리 (중복 제거 + 품질 필터링)"""
    print("=" * 60)
    print("데이터 전처리 중... (v2: 강화된 버전)")
    
    eng_processed = [preprocess_sentence(s, 'eng') for s in tqdm(eng_sentences, desc="영어")]
    kor_processed = [preprocess_sentence(s, 'kor') for s in tqdm(kor_sentences, desc="한국어")]
    
    filtered_pairs = []
    seen_pairs = set()
    stats = {'length': 0, 'duplicate': 0, 'no_text': 0, 'too_short': 0, 'ratio': 0}
    
    for e, k in zip(eng_processed, kor_processed):
        e_words, k_words = len(e.split()), len(k.split())
        
        if not (3 <= e_words <= 50 and 2 <= k_words <= 60):
            stats['length'] += 1
            continue
        
        pair_key = (k, e)
        if pair_key in seen_pairs:
            stats['duplicate'] += 1
            continue
        seen_pairs.add(pair_key)
        
        if not re.search(r'[가-힣]', k) or not re.search(r'[a-z]', e):
            stats['no_text'] += 1
            continue
        
        if len(k) < 5 or len(e) < 10:
            stats['too_short'] += 1
            continue
        
        ratio = e_words / k_words if k_words > 0 else 0
        if not (0.5 <= ratio <= 3.0):
            stats['ratio'] += 1
            continue
        
        filtered_pairs.append((e, k))
    
    eng_filtered, kor_filtered = zip(*filtered_pairs) if filtered_pairs else ([], [])
    eng_filtered, kor_filtered = list(eng_filtered), list(kor_filtered)
    
    original = len(eng_sentences)
    final = len(eng_filtered)
    print(f"\n📊 전처리 결과: {original:,} → {final:,} ({final/original*100:.1f}%)")
    print(f"제거: 길이({stats['length']}), 중복({stats['duplicate']}), 비율({stats['ratio']})")
    print("=" * 60)
    
    if show_samples > 0:
        print(f"\n📋 샘플 ({show_samples}개)")
        for i in random.sample(range(len(eng_filtered)), min(show_samples, len(eng_filtered))):
            print(f"🇰🇷 {kor_filtered[i][:50]}...")
            print(f"🇺🇸 {eng_filtered[i][:50]}...\n")
    
    return eng_filtered, kor_filtered

# 전처리 실행
eng_filtered, kor_filtered = preprocess_data_v2(answers, questions, show_samples=2)

# 5. 데이터 증강 (30%)

In [ ]:
# KoEDA 증강
try:
    from koeda import EDA
    
    class KoEDAugmenter:
        def __init__(self):
            print("KoEDA 초기화 중...")
            self.eda = EDA(morpheme_analyzer="Okt", alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, prob_rd=0.1)
            print("✅ KoEDA 초기화 완료!")

        def augment(self, sentence, num_aug=1):
            if len(sentence.split()) <= 1:
                return []
            try:
                result = self.eda(sentence)
                augmented = [result] if isinstance(result, str) else result
                return [s for s in augmented if s != sentence and len(s.strip()) > 0]
            except:
                return []
    
    KOEDA_AVAILABLE = True
except:
    KOEDA_AVAILABLE = False
    print("⚠️ KoEDA 미설치")

# 데이터 분할
train_ko, temp_ko, train_en, temp_en = train_test_split(
    kor_filtered, eng_filtered, test_size=0.2, random_state=SEED
)
val_ko, test_ko, val_en, test_en = train_test_split(
    temp_ko, temp_en, test_size=0.5, random_state=SEED
)

print(f"데이터 분할: Train={len(train_ko):,}, Val={len(val_ko):,}, Test={len(test_ko):,}")

# 30% 증강
if KOEDA_AVAILABLE:
    AUGMENT_RATIO = 0.3
    AUGMENT_NEEDED = int(len(train_ko) * AUGMENT_RATIO)
    
    print(f"\n증강 목표: {AUGMENT_NEEDED:,}개 (30%)")
    
    augmenter = KoEDAugmenter()
    
    # 증강할 샘플 선택
    aug_indices = random.sample(range(len(train_ko)), min(AUGMENT_NEEDED, len(train_ko)))
    
    aug_ko, aug_en = [], []
    for idx in tqdm(aug_indices, desc="증강 중"):
        augmented = augmenter.augment(train_ko[idx])
        if augmented:
            aug_ko.append(augmented[0])
            aug_en.append(train_en[idx])
    
    # 원본 + 증강
    train_ko = list(train_ko) + aug_ko
    train_en = list(train_en) + aug_en
    
    # 셔플
    combined = list(zip(train_ko, train_en))
    random.shuffle(combined)
    train_ko, train_en = zip(*combined)
    train_ko, train_en = list(train_ko), list(train_en)
    
    print(f"\n✅ 증강 완료! Train: {len(train_ko):,}개")

# 저장
with open(KOR_TRAIN_PATH, 'w', encoding='utf-8') as f:
    for s in train_ko: f.write(s + '\n')
with open(ENG_TRAIN_PATH, 'w', encoding='utf-8') as f:
    for s in train_en: f.write(s + '\n')

print(f"📁 저장 완료: {KOR_TRAIN_PATH}")

# 6. 토크나이저 정의 및 학습

In [ ]:
def train_sentencepiece(data, model_prefix, vocab_size=8000):
    print(f"SentencePiece 학습: {model_prefix}")
    corpus_path = f'{model_prefix}_corpus.txt'
    with open(corpus_path, 'w', encoding='utf-8') as f:
        for line in data:
            f.write(f'{line}\n')
    
    spm.SentencePieceTrainer.Train(
        f'--input={corpus_path} --model_prefix={model_prefix} --vocab_size={vocab_size} '
        f'--model_type=unigram --pad_id=0 --unk_id=1 --bos_id=2 --eos_id=3 --character_coverage=1.0'
    )
    os.remove(corpus_path)
    print(f"✅ 완료: {model_prefix}.model")


class SentencePieceVocab:
    def __init__(self, model_path):
        self.sp = spm.SentencePieceProcessor()
        self.sp.Load(model_path)
        self.PAD_ID, self.UNK_ID, self.BOS_ID, self.EOS_ID = 0, 1, 2, 3
        self.itos = [self.sp.IdToPiece(i) for i in range(self.sp.GetPieceSize())]

    def encode(self, text): return self.sp.EncodeAsIds(text)
    def decode(self, ids): return self.sp.DecodeIds([i for i in ids if i not in [0,2,3]])
    def __len__(self): return self.sp.GetPieceSize()


# 토크나이저 학습/로드
kor_model_path = f"{KOR_TOKENIZER_PREFIX}.model"
eng_model_path = f"{ENG_TOKENIZER_PREFIX}.model"

if os.path.exists(kor_model_path) and os.path.exists(eng_model_path):
    print("기존 토크나이저 로드 중...")
else:
    print("토크나이저 학습 중...")
    all_kor = train_ko + list(val_ko) + list(test_ko)
    all_eng = train_en + list(val_en) + list(test_en)
    train_sentencepiece(all_kor, KOR_TOKENIZER_PREFIX, DATA_CONFIG.vocab_size_kor)
    train_sentencepiece(all_eng, ENG_TOKENIZER_PREFIX, DATA_CONFIG.vocab_size_eng)

vocab_src = SentencePieceVocab(kor_model_path)
vocab_trg = SentencePieceVocab(eng_model_path)

print(f"✅ 한국어 Vocab: {len(vocab_src):,}, 영어 Vocab: {len(vocab_trg):,}")

# 7. Dataset 및 DataLoader

In [ ]:
class TranslationDataset(Dataset):
    def __init__(self, src_list, trg_list, vocab_src, vocab_trg, max_length=60):
        self.src_list, self.trg_list = src_list, trg_list
        self.vocab_src, self.vocab_trg = vocab_src, vocab_trg
        self.max_length = max_length

    def __len__(self): return len(self.src_list)

    def __getitem__(self, idx):
        src = [self.vocab_src.BOS_ID] + self.vocab_src.encode(self.src_list[idx]) + [self.vocab_src.EOS_ID]
        trg = [self.vocab_trg.BOS_ID] + self.vocab_trg.encode(self.trg_list[idx]) + [self.vocab_trg.EOS_ID]
        return {
            'SRC': torch.tensor(src[:self.max_length], dtype=torch.long),
            'TRG': torch.tensor(trg[:self.max_length], dtype=torch.long),
            'SRC_TEXT': self.src_list[idx], 'TRG_TEXT': self.trg_list[idx]
        }

def collate_fn(batch, pad_idx=0):
    src = pad_sequence([item['SRC'] for item in batch], batch_first=True, padding_value=pad_idx)
    trg = pad_sequence([item['TRG'] for item in batch], batch_first=True, padding_value=pad_idx)
    return {'SRC': src, 'TRG': trg, 'SRC_TEXT': [item['SRC_TEXT'] for item in batch], 'TRG_TEXT': [item['TRG_TEXT'] for item in batch]}

# DataLoader 생성
train_dataset = TranslationDataset(train_ko, train_en, vocab_src, vocab_trg, DATA_CONFIG.max_length)
val_dataset = TranslationDataset(list(val_ko), list(val_en), vocab_src, vocab_trg, DATA_CONFIG.max_length)
test_dataset = TranslationDataset(list(test_ko), list(test_en), vocab_src, vocab_trg, DATA_CONFIG.max_length)

collate_with_pad = lambda batch: collate_fn(batch, pad_idx=vocab_src.PAD_ID)

train_loader = DataLoader(train_dataset, batch_size=DATA_CONFIG.batch_size, shuffle=True, collate_fn=collate_with_pad)
val_loader = DataLoader(val_dataset, batch_size=DATA_CONFIG.batch_size, shuffle=False, collate_fn=collate_with_pad)
test_loader = DataLoader(test_dataset, batch_size=DATA_CONFIG.batch_size, shuffle=False, collate_fn=collate_with_pad)

print(f"✅ DataLoader: Train={len(train_loader)}, Val={len(val_loader)}, Test={len(test_loader)} batches")

# 8. Transformer 모델 정의

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, emb_dim, num_heads, dropout=0.0, bias=False, encoder_decoder_attention=False, causal=False):
        super().__init__()
        self.emb_dim, self.num_heads = emb_dim, num_heads
        self.dropout, self.head_dim = dropout, emb_dim // num_heads
        self.encoder_decoder_attention, self.causal = encoder_decoder_attention, causal
        self.q_proj = nn.Linear(emb_dim, emb_dim, bias=bias)
        self.k_proj = nn.Linear(emb_dim, emb_dim, bias=bias)
        self.v_proj = nn.Linear(emb_dim, emb_dim, bias=bias)
        self.out_proj = nn.Linear(emb_dim, emb_dim, bias=bias)

    def transpose_for_scores(self, x):
        return x.view(*x.size()[:-1], self.num_heads, self.head_dim).permute(0, 2, 1, 3)

    def forward(self, query, key, attention_mask=None):
        q = self.transpose_for_scores(self.q_proj(query))
        k = self.transpose_for_scores(self.k_proj(key if self.encoder_decoder_attention else query))
        v = self.transpose_for_scores(self.v_proj(key if self.encoder_decoder_attention else query))
        
        attn_weights = torch.matmul(q, k.transpose(-1, -2)) / math.sqrt(self.head_dim)
        if attention_mask is not None:
            if self.causal:
                attn_weights = attn_weights.masked_fill(attention_mask.unsqueeze(0).unsqueeze(1), float("-inf"))
            else:
                attn_weights = attn_weights.masked_fill(attention_mask.unsqueeze(1).unsqueeze(2), float("-inf"))
        
        attn_probs = F.dropout(F.softmax(attn_weights, dim=-1), p=self.dropout, training=self.training)
        attn_output = torch.matmul(attn_probs, v).permute(0, 2, 1, 3).contiguous()
        return self.out_proj(attn_output.view(*attn_output.size()[:-2], self.emb_dim)), attn_weights


class PositionWiseFeedForward(nn.Module):
    def __init__(self, emb_dim, d_ff, dropout=0.1):
        super().__init__()
        self.w_1, self.w_2 = nn.Linear(emb_dim, d_ff), nn.Linear(d_ff, emb_dim)
        self.dropout = dropout

    def forward(self, x):
        return F.dropout(self.w_2(F.dropout(F.relu(self.w_1(x)), p=self.dropout, training=self.training)), p=self.dropout, training=self.training)


class SinusoidalPositionalEmbedding(nn.Embedding):
    def __init__(self, num_positions, embedding_dim, padding_idx=None):
        super().__init__(num_positions, embedding_dim)
        position_enc = torch.zeros(num_positions, embedding_dim)
        for pos in range(num_positions):
            for i in range(0, embedding_dim, 2):
                position_enc[pos, i] = math.sin(pos / (10000 ** (i / embedding_dim)))
                if i + 1 < embedding_dim:
                    position_enc[pos, i + 1] = math.cos(pos / (10000 ** ((i + 1) / embedding_dim)))
        self.weight.data.copy_(position_enc)
        self.weight.requires_grad = False

    @torch.no_grad()
    def forward(self, input_ids):
        return super().forward(torch.arange(input_ids.size(1), device=self.weight.device))


class EncoderLayer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.self_attn = MultiHeadAttention(config.emb_dim, config.num_heads, config.attention_dropout)
        self.ffn = PositionWiseFeedForward(config.emb_dim, config.ffn_dim, config.dropout)
        self.norm1, self.norm2 = nn.LayerNorm(config.emb_dim), nn.LayerNorm(config.emb_dim)
        self.dropout = config.dropout

    def forward(self, x, mask=None):
        attn_out, attn_weights = self.self_attn(self.norm1(x), self.norm1(x), mask)
        x = x + F.dropout(attn_out, p=self.dropout, training=self.training)
        x = x + F.dropout(self.ffn(self.norm2(x)), p=self.dropout, training=self.training)
        return x, attn_weights


class DecoderLayer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.self_attn = MultiHeadAttention(config.emb_dim, config.num_heads, config.attention_dropout, causal=True)
        self.cross_attn = MultiHeadAttention(config.emb_dim, config.num_heads, config.attention_dropout, encoder_decoder_attention=True)
        self.ffn = PositionWiseFeedForward(config.emb_dim, config.ffn_dim, config.dropout)
        self.norm1, self.norm2, self.norm3 = nn.LayerNorm(config.emb_dim), nn.LayerNorm(config.emb_dim), nn.LayerNorm(config.emb_dim)
        self.dropout = config.dropout

    def forward(self, x, enc_out, enc_mask=None, dec_mask=None):
        self_attn_out, self_attn_w = self.self_attn(self.norm1(x), self.norm1(x), dec_mask)
        x = x + F.dropout(self_attn_out, p=self.dropout, training=self.training)
        cross_attn_out, cross_attn_w = self.cross_attn(self.norm2(x), enc_out, enc_mask)
        x = x + F.dropout(cross_attn_out, p=self.dropout, training=self.training)
        x = x + F.dropout(self.ffn(self.norm3(x)), p=self.dropout, training=self.training)
        return x, (self_attn_w, cross_attn_w)


class Encoder(nn.Module):
    def __init__(self, config, embed_tokens):
        super().__init__()
        self.embed_tokens = embed_tokens
        self.embed_positions = SinusoidalPositionalEmbedding(config.max_position_embeddings, config.emb_dim)
        self.layers = nn.ModuleList([EncoderLayer(config) for _ in range(config.encoder_layers)])
        self.dropout = nn.Dropout(config.dropout)
        self.final_norm = nn.LayerNorm(config.emb_dim)

    def forward(self, input_ids, mask=None):
        x = self.dropout(self.embed_tokens(input_ids) + self.embed_positions(input_ids))
        attn_scores = []
        for layer in self.layers:
            x, attn = layer(x, mask)
            attn_scores.append(attn)
        return self.final_norm(x), attn_scores


class Decoder(nn.Module):
    def __init__(self, config, embed_tokens):
        super().__init__()
        self.embed_tokens = embed_tokens
        self.embed_positions = SinusoidalPositionalEmbedding(config.max_position_embeddings, config.emb_dim)
        self.layers = nn.ModuleList([DecoderLayer(config) for _ in range(config.decoder_layers)])
        self.dropout = nn.Dropout(config.dropout)
        self.final_norm = nn.LayerNorm(config.emb_dim)

    def forward(self, input_ids, enc_out, enc_mask=None, dec_mask=None):
        x = self.dropout(self.embed_tokens(input_ids) + self.embed_positions(input_ids))
        attn_scores = []
        for layer in self.layers:
            x, attn = layer(x, enc_out, enc_mask, dec_mask)
            attn_scores.append(attn)
        return self.final_norm(x), attn_scores


class Transformer(nn.Module):
    def __init__(self, vocab_src, vocab_trg, config):
        super().__init__()
        self.vocab_src, self.vocab_trg, self.config = vocab_src, vocab_trg, config
        self.enc_embedding = nn.Embedding(len(vocab_src), config.emb_dim, padding_idx=vocab_src.PAD_ID)
        self.dec_embedding = nn.Embedding(len(vocab_trg), config.emb_dim, padding_idx=vocab_trg.PAD_ID)
        self.encoder = Encoder(config, self.enc_embedding)
        self.decoder = Decoder(config, self.dec_embedding)
        self.prediction_head = nn.Linear(config.emb_dim, len(vocab_trg))
        self._init_weights()

    def _init_weights(self):
        for p in self.parameters():
            if p.dim() > 1: nn.init.xavier_uniform_(p)

    def forward(self, src, trg):
        enc_mask = (src == self.vocab_src.PAD_ID)
        dec_mask = torch.triu(torch.ones(trg.size(1), trg.size(1), dtype=torch.bool, device=trg.device), diagonal=1)
        enc_out, enc_attn = self.encoder(src, enc_mask)
        dec_out, dec_attn = self.decoder(trg, enc_out, enc_mask, dec_mask)
        return self.prediction_head(dec_out), enc_attn, dec_attn

    def count_parameters(self):
        return sum(p.numel() for p in self.parameters() if p.requires_grad)

print("✅ Transformer 모델 정의 완료!")

# 9. 손실 함수 및 ⭐ Warmup Scheduler (핵심!)

In [ ]:
class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, smoothing=0.1, dim=-1, ignore_index=None):
        super().__init__()
        self.confidence, self.smoothing = 1.0 - smoothing, smoothing
        self.cls, self.dim, self.ignore_index = classes, dim, ignore_index

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=self.dim)
        with torch.no_grad():
            true_dist = torch.zeros_like(pred).fill_(self.smoothing / (self.cls - 2))
            true_dist.scatter_(1, target.unsqueeze(1), self.confidence)
            if self.ignore_index is not None:
                true_dist[:, self.ignore_index] = 0
                mask = torch.nonzero(target == self.ignore_index)
                if mask.dim() > 0: true_dist.index_fill_(0, mask.squeeze(), 0.0)
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))


class TransformerScheduler:
    """
    ⭐ Attention Is All You Need 논문의 Learning Rate Schedule
    lr = d_model^(-0.5) * min(step^(-0.5), step * warmup_steps^(-1.5))
    """
    def __init__(self, optimizer, d_model, warmup_steps=4000):
        self.optimizer = optimizer
        self.d_model = d_model
        self.warmup_steps = warmup_steps
        self.step_num = 0
        self.current_lr = 0
        
    def step(self):
        self.step_num += 1
        self.current_lr = (self.d_model ** -0.5) * min(self.step_num ** -0.5, self.step_num * (self.warmup_steps ** -1.5))
        for param_group in self.optimizer.param_groups:
            param_group['lr'] = self.current_lr
        return self.current_lr
    
    def get_last_lr(self):
        return [self.current_lr]

print("✅ Loss 및 TransformerScheduler 정의 완료!")

# 10. 학습 및 평가 함수

In [ ]:
def train_epoch_improved(model, iterator, optimizer, scheduler, criterion, clip, device):
    """개선된 학습 함수 (Scheduler 포함)"""
    model.train()
    epoch_loss = 0
    progress_bar = tqdm(iterator, desc="Training", leave=False)
    
    for batch in progress_bar:
        src, trg = batch['SRC'].to(device), batch['TRG'].to(device)
        optimizer.zero_grad()
        
        output, _, _ = model(src, trg[:, :-1])
        output = output.contiguous().view(-1, output.shape[-1])
        trg_out = trg[:, 1:].contiguous().view(-1)
        
        loss = criterion(output, trg_out)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        current_lr = scheduler.step()  # ⭐ 매 배치마다 LR 업데이트
        
        epoch_loss += loss.item()
        progress_bar.set_postfix(loss=f"{loss.item():.3f}", lr=f"{current_lr:.2e}")
    
    return epoch_loss / len(iterator)


def evaluate(model, iterator, criterion, device):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for batch in iterator:
            src, trg = batch['SRC'].to(device), batch['TRG'].to(device)
            output, _, _ = model(src, trg[:, :-1])
            output = output.contiguous().view(-1, output.shape[-1])
            trg_out = trg[:, 1:].contiguous().view(-1)
            epoch_loss += criterion(output, trg_out).item()
    return epoch_loss / len(iterator)

print("✅ 학습/평가 함수 정의 완료!")

# 11. 번역 함수 (Greedy + Beam Search)

In [ ]:
def translate_sentence(sentence, model, vocab_src, vocab_trg, device, max_len=60, repetition_penalty=1.3, no_repeat_ngram=3):
    """Greedy 번역 + Repetition Penalty"""
    model.eval()
    tokens = [vocab_src.BOS_ID] + vocab_src.encode(sentence) + [vocab_src.EOS_ID]
    src_tensor = torch.LongTensor(tokens).unsqueeze(0).to(device)
    src_mask = (src_tensor == vocab_src.PAD_ID)

    with torch.no_grad():
        enc_out, _ = model.encoder(src_tensor, src_mask)
        trg_indexes = [vocab_trg.BOS_ID]

        for _ in range(max_len):
            trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device)
            dec_mask = torch.triu(torch.ones(len(trg_indexes), len(trg_indexes), dtype=torch.bool, device=device), diagonal=1)
            dec_out, dec_attn = model.decoder(trg_tensor, enc_out, src_mask, dec_mask)
            logits = dec_out[:, -1, :]
            
            # Repetition Penalty
            for prev in set(trg_indexes):
                logits[0, prev] = logits[0, prev] / repetition_penalty if logits[0, prev] > 0 else logits[0, prev] * repetition_penalty
            
            # No Repeat N-gram
            if no_repeat_ngram > 0 and len(trg_indexes) >= no_repeat_ngram:
                prefix = tuple(trg_indexes[-(no_repeat_ngram-1):])
                for j in range(len(trg_indexes) - no_repeat_ngram + 1):
                    if tuple(trg_indexes[j:j+no_repeat_ngram-1]) == prefix:
                        logits[0, trg_indexes[j+no_repeat_ngram-1]] = float('-inf')
            
            pred = logits.argmax(-1).item()
            trg_indexes.append(pred)
            if pred == vocab_trg.EOS_ID: break

    return [vocab_trg.itos[i] for i in trg_indexes[1:]], dec_attn[-1][1].squeeze(0).cpu().numpy()


def translate_beam_search(sentence, model, vocab_src, vocab_trg, device, beam_size=5, max_len=60, length_penalty=0.6):
    """Beam Search 번역 (더 좋은 품질)"""
    model.eval()
    tokens = [vocab_src.BOS_ID] + vocab_src.encode(sentence) + [vocab_src.EOS_ID]
    src_tensor = torch.LongTensor(tokens).unsqueeze(0).to(device)
    src_mask = (src_tensor == vocab_src.PAD_ID)

    with torch.no_grad():
        enc_out, _ = model.encoder(src_tensor, src_mask)
        beams = [(0.0, [vocab_trg.BOS_ID])]
        completed = []

        for _ in range(max_len):
            candidates = []
            for score, seq in beams:
                if seq[-1] == vocab_trg.EOS_ID:
                    completed.append((score / (len(seq) ** length_penalty), seq))
                    continue
                trg_tensor = torch.LongTensor(seq).unsqueeze(0).to(device)
                dec_mask = torch.triu(torch.ones(len(seq), len(seq), dtype=torch.bool, device=device), diagonal=1)
                dec_out, _ = model.decoder(trg_tensor, enc_out, src_mask, dec_mask)
                log_probs = F.log_softmax(dec_out[:, -1, :], dim=-1)
                top_probs, top_ids = log_probs.topk(beam_size)
                for prob, idx in zip(top_probs[0], top_ids[0]):
                    candidates.append((score + prob.item(), seq + [idx.item()]))
            beams = sorted(candidates, key=lambda x: x[0], reverse=True)[:beam_size]
            if len(completed) >= beam_size: break

        for s, seq in beams:
            completed.append((s / (len(seq) ** length_penalty), seq))
        best = max(completed, key=lambda x: x[0]) if completed else beams[0]

    return [vocab_trg.itos[i] for i in best[1] if i != vocab_trg.BOS_ID]

print("✅ 번역 함수 정의 완료!")

# 12. 🚀 모델 초기화 및 학습

In [ ]:
# 모델 초기화
print("=" * 60)
print("🔧 모델 초기화 (개선된 설정)")
print("=" * 60)

model = Transformer(vocab_src, vocab_trg, MODEL_CONFIG).to(device)
print(f"모델 파라미터 수: {model.count_parameters():,}")
print(f"d_model: {MODEL_CONFIG.emb_dim}, ffn_dim: {MODEL_CONFIG.ffn_dim}")

# 옵티마이저 (lr=0으로 시작, Scheduler가 관리)
optimizer = optim.Adam(
    model.parameters(), lr=0,
    betas=TRAIN_CONFIG.betas, eps=TRAIN_CONFIG.eps, weight_decay=TRAIN_CONFIG.weight_decay
)

# ⭐ Warmup Scheduler
scheduler = TransformerScheduler(optimizer, d_model=MODEL_CONFIG.emb_dim, warmup_steps=TRAIN_CONFIG.warmup_steps)

# 손실 함수
criterion = LabelSmoothingLoss(classes=len(vocab_trg), smoothing=TRAIN_CONFIG.label_smoothing, ignore_index=vocab_trg.PAD_ID).to(device)

print(f"Warmup Steps: {TRAIN_CONFIG.warmup_steps}")
print("=" * 60)

In [ ]:
# 학습 실행
print("\n" + "=" * 60)
print("🚀 학습 시작!")
print("=" * 60)

train_losses, val_losses, learning_rates = [], [], []
best_valid_loss = float('inf')
patience_counter = 0
total_start_time = time.time()

for epoch in range(TRAIN_CONFIG.num_epochs):
    epoch_start = time.time()
    
    train_loss = train_epoch_improved(model, train_loader, optimizer, scheduler, criterion, TRAIN_CONFIG.clip, device)
    val_loss = evaluate(model, val_loader, criterion, device)
    
    current_lr = scheduler.get_last_lr()[0]
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    learning_rates.append(current_lr)
    
    epoch_time = time.time() - epoch_start
    print(f"[Epoch {epoch+1:02d}/{TRAIN_CONFIG.num_epochs}] Train: {train_loss:.4f} | Val: {val_loss:.4f} | PPL: {np.exp(val_loss):.2f} | LR: {current_lr:.2e} | {epoch_time/60:.1f}m", end="")
    
    if val_loss < best_valid_loss - TRAIN_CONFIG.min_delta:
        best_valid_loss = val_loss
        patience_counter = 0
        torch.save({'epoch': epoch, 'model_state_dict': model.state_dict(), 'val_loss': val_loss, 'config': MODEL_CONFIG}, MODEL_SAVE_PATH)
        print(" ✅ Saved!")
    else:
        patience_counter += 1
        print(f" (patience: {patience_counter}/{TRAIN_CONFIG.patience})")
    
    if patience_counter >= TRAIN_CONFIG.patience:
        print(f"\n⚠️ Early Stopping at Epoch {epoch+1}")
        break

print("\n" + "=" * 60)
print(f"✅ 학습 완료! 총 시간: {(time.time()-total_start_time)/60:.1f}분")
print(f"Best Val Loss: {best_valid_loss:.4f}, PPL: {np.exp(best_valid_loss):.2f}")
print("=" * 60)

# 13. 학습 결과 시각화

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

axes[0].plot(train_losses, label='Train', color='blue')
axes[0].plot(val_losses, label='Val', color='red')
axes[0].set_xlabel('Epoch'); axes[0].set_ylabel('Loss')
axes[0].set_title('Loss'); axes[0].legend(); axes[0].grid(True, alpha=0.3)

axes[1].plot([np.exp(l) for l in train_losses], label='Train', color='blue')
axes[1].plot([np.exp(l) for l in val_losses], label='Val', color='red')
axes[1].set_xlabel('Epoch'); axes[1].set_ylabel('Perplexity')
axes[1].set_title('Perplexity'); axes[1].legend(); axes[1].grid(True, alpha=0.3)

axes[2].plot(learning_rates, color='green')
axes[2].set_xlabel('Epoch'); axes[2].set_ylabel('LR')
axes[2].set_title('Learning Rate (Warmup)'); axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{MODEL_DIR}/training_curves.png', dpi=150)
plt.show()

# 14. 번역 테스트

In [ ]:
# Best 모델 로드
checkpoint = torch.load(MODEL_SAVE_PATH, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

print("=" * 60)
print("🔤 번역 테스트")
print("=" * 60)

test_sentences = [
    "북한이 미사일을 발사했다.",
    "대통령이 기자회견을 열었다.",
    "주식 시장이 하락했다.",
    "미국과 중국의 무역 전쟁이 계속되고 있다.",
]

for i, sent in enumerate(test_sentences, 1):
    print(f"\n[{i}] 원문: {sent}")
    
    greedy, _ = translate_sentence(sent, model, vocab_src, vocab_trg, device)
    greedy_text = ''.join([t for t in greedy if t not in ['<eos>','</s>']]).replace('▁', ' ').strip()
    print(f"    Greedy: {greedy_text}")
    
    beam = translate_beam_search(sent, model, vocab_src, vocab_trg, device, beam_size=5)
    beam_text = ''.join([t for t in beam if t not in ['<eos>','</s>']]).replace('▁', ' ').strip()
    print(f"    Beam:   {beam_text}")

print("\n" + "=" * 60)
print("✅ 번역 테스트 완료!")

In [ ]:
# 테스트셋 샘플 번역
print("\n📋 테스트셋 샘플")
print("-" * 60)

for i in range(5):
    src = test_dataset.src_list[i]
    trg = test_dataset.trg_list[i]
    beam = translate_beam_search(src, model, vocab_src, vocab_trg, device, beam_size=5)
    pred = ''.join([t for t in beam if t not in ['<eos>','</s>']]).replace('▁', ' ').strip()
    
    print(f"\n[{i+1}]")
    print(f"  원문: {src[:60]}...")
    print(f"  정답: {trg[:60]}...")
    print(f"  예측: {pred[:60]}...")

# ✅ 완료!

## 개선 효과 요약

| 지표 | 기존 | 개선 후 |
|------|------|--------|
| Val Loss | ~2.93 | **< 2.0** (예상) |
| Perplexity | ~18.7 | **< 8** (예상) |
| 번역 품질 | 의미 없음 | **의미 있는 문장** |

핵심 개선: **Warmup Learning Rate Scheduler**